<a href="https://colab.research.google.com/github/hyunicecream/ML-DL/blob/main/LSTM%2BCNN%20%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%20%EC%A3%BC%EA%B0%80%20%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.layers import Dense, Input, LSTM, Average
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# LSTM과 CNN을 결합한 모델을 이용해서 주가 수익률을 예측한다.
# 주체별 매매 동향 데이터로 향후 주가 수익률을 추정한다.
%cd '/content/drive/MyDrive/Private_room/data'
data = pd.read_csv('삼성전자.csv')
data.head()

/content/drive/MyDrive/Private_room/data


,날짜,종가,전일비,등락율,거래량,개인,기관,외국인,금융투자,보험,투신,은행,기타금융,연기금,사모펀드,국가지자체,기타법인,기타외국인,개인누적,기관누적,외국인누적,금투누적,투신누적,연기금누적
0,2020.6.1,51200,500,NaN,16289568,-1859511,2104912,-39405,795040,274522,93773,17922,-34554,921107,37102,0,-190526,-15470,-1859511,2104912,-39405,795040,93773,921107
1,2020.6.2,51400,200,0.003906,13764485,-560690,873157,-357253,81149,568419,48140,7572,-1260,108237,60900,0,24180,20606,-2420201,2978069,-396658,876189,141913,1029344
2,2020.6.3,54500,3100,0.060311,48817627,-12644642,9757786,3271861,6464027,1568929,1297994,-54218,-28639,42381,467312,0,-354941,-30064,-15064843,12735855,2875203,7340216,1439907,1071725
3,2020.6.4,54600,100,0.001835,40088343,-3179569,1604890,1678893,2894351,-297073,279363,-9337,28892,-1572203,280897,0,-70151,-34063,-18244412,14340745,4554096,10234567,1719270,-500478
4,2020.6.5,55500,900,0.016484,22402378,-2638431,2869587,-184385,2291970,108763,1044160,6651,-18200,-525779,-37978,0,-25417,-21354,-20882843,17210332,4369711,12526537,2763430,-1026257


In [ ]:
last_price = list(data['종가'])[-1]
last_price

80000

In [ ]:
# 결측치 제거 why? NaN값을 삭제하기 위해서
df = data.dropna()
df = df.drop(['날짜', '종가', '전일비', '개인누적', '기관누적', '외국인누적', '금투누적', '투신누적', '연기금누적', '국가지자체'], axis=1)
df.head()

,등락율,거래량,개인,기관,외국인,금융투자,보험,투신,은행,기타금융,연기금,사모펀드,기타법인,기타외국인
1,0.003906,13764485,-560690,873157,-357253,81149,568419,48140,7572,-1260,108237,60900,24180,20606
2,0.060311,48817627,-12644642,9757786,3271861,6464027,1568929,1297994,-54218,-28639,42381,467312,-354941,-30064
3,0.001835,40088343,-3179569,1604890,1678893,2894351,-297073,279363,-9337,28892,-1572203,280897,-70151,-34063
4,0.016484,22402378,-2638431,2869587,-184385,2291970,108763,1044160,6651,-18200,-525779,-37978,-25417,-21354
5,-0.010811,24555972,2852709,-1098871,-1650246,-723060,-67082,728799,32155,53610,-1144027,20734,-121115,17523


In [ ]:
# feature값 표준화
rtn_mean = df['등락율'].mean() 
# 복원을 위해 저장해 둔다.
# 마지막에 주가 예측을 위해 사용하기 위해서
# 주가 = 마지막종가 X (1 + 수익율)
rtn_std = df['등락율'].std()
df = (df - df.mean()) / df.std()


,등락율,거래량,개인,기관,외국인,금융투자,보험,투신,은행,기타금융,연기금,사모펀드,기타법인,기타외국인
1,0.114683,-0.775279,-0.391224,0.698600,0.028886,0.098217,3.295896,0.523499,0.232549,0.024333,0.908907,0.568486,-0.013173,0.882745
2,3.382213,2.903239,-3.004852,4.353241,1.120881,3.631036,8.439133,4.822001,-0.984074,-0.937201,0.812528,3.027572,-2.500987,-1.832060
3,-0.005312,1.987178,-0.957659,0.999594,0.641559,1.655278,-1.153265,1.318722,-0.100383,1.083252,-1.550371,1.899627,-0.632178,-2.046319
4,0.843281,0.131192,-0.840617,1.519820,0.080901,1.321870,0.932981,3.949014,0.214415,-0.570589,-0.018958,-0.029798,-0.338631,-1.365394
5,-0.737873,0.357192,0.347057,-0.112582,-0.360174,-0.346900,0.029030,2.864424,0.716580,1.951333,-0.923747,0.325452,-0.966607,0.717563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0.532854,-0.688346,-0.860713,0.302894,0.953605,0.030575,0.143871,0.298256,-0.072935,0.164213,0.704997,0.506448,0.581206,-0.785411
248,-0.748975,-0.802738,0.676183,-0.540155,-0.552534,-0.824944,0.436614,-0.029525,0.243753,0.083052,0.231988,-0.437471,0.037749,0.931340
249,-0.398032,-0.685061,0.797621,0.563401,-1.546361,0.513428,-0.651161,0.581673,-0.011957,0.043648,0.559045,0.102471,0.451506,0.692703
250,-0.543380,-0.536417,1.379629,-1.012567,-1.148151,-1.062934,0.045094,0.272190,-0.051493,-0.064168,-0.721585,-0.796104,-0.761306,0.789090


In [ ]:
# 2차원 배열의 feature 데이터로 LSTM 학습 데이터를 만든다.
def build_train_data(data, t_step, n_jump = 1):
    n_data = data.shape[0]   # number of data
    n_feat = data.shape[1]   # number of features

    m = np.arange(0, n_data - t_step, n_jump)   # m =  [0, 1, 2, 3, 4, 5]
    x = [data[i:(i+t_step), :] for i in m]      # feature data
    y = [data[i, :] for i in (m + t_step)]      # target data

    # shape을 조정한다. feature = 3D, target = 2D
    x_data = np.reshape(np.array(x), (len(m), t_step, n_feat))
    y_target = np.reshape(np.array(y), (len(m), n_feat))
    
    return x_data, y_target

In [ ]:
t_step = 20

# 학습 데이터를 생성한다.
x_train, y_train = build_train_data(np.array(df), t_step, n_jump=1)
x_train.shape, y_train.shape

((231, 20, 14), (231, 14))

In [ ]:
n_feat = x_train.shape[2]
n_output = y_train.shape[1]
n_hidden = 128

# LSTM 모델을 생성한다.
x_lstm_input = Input(batch_shape=(None, t_step, n_feat))
x_lstm = LSTM(n_hidden, return_sequences = True)(x_lstm_input)
x_lstm = LSTM(n_hidden, dropout=0.2)(x_lstm)

# CNN 모델을 생성한다. 4D
# CONV2D : input 2개, CONV1D : input 1개
x_cnn_input = Input(batch_shape = (None, t_step, n_feat, 1))
x_conv = Conv2D(filters=10, kernel_size=(20,5), strides=1, padding = 'same', activation='relu')(x_cnn_input)
x_pool = MaxPooling2D(pool_size=(10,5), strides=1, padding='same')(x_conv)
x_flat = Flatten()(x_pool)
x_cnn = Dense(n_hidden)(x_flat) #LSTM 출력크기와 맞춘다

# 두 네트워크를 합친다.
x_avg = Average()([x_lstm, x_cnn])
y_output = Dense(n_feat)(x_avg)

model = Model([x_lstm_input, x_cnn_input], y_output)
model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 20, 14, 1)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 20, 14, 10)   1010        ['input_6[0][0]']                
                                                                                                  
 input_5 (InputLayer)           [(None, 20, 14)]     0           []                               
                                                                                                  
 max_pooling2d_1 (MaxPooling2D)  (None, 20, 14, 10)  0           ['conv2d_1[0][0]']               
                                                                                            

In [ ]:
# LSTM과 CNN은 차원이 다르기 때문에 reshape으로 변환해준다.
h = model.fit([x_train, x_train.reshape(-1, t_step, n_feat, 1)], y_train, epochs=50, batch_size=32, shuffle=True)

Epoch 1/50
8/8 [==============================] - 8s 118ms/step - loss: 4.4215
Epoch 2/50
8/8 [==============================] - 1s 112ms/step - loss: 1.8376
Epoch 3/50
8/8 [==============================] - 1s 116ms/step - loss: 1.2233
Epoch 4/50
8/8 [==============================] - 1s 117ms/step - loss: 0.9242
Epoch 5/50
8/8 [==============================] - 1s 116ms/step - loss: 0.8392
Epoch 6/50
8/8 [==============================] - 1s 113ms/step - loss: 0.8227
Epoch 7/50
8/8 [==============================] - 1s 109ms/step - loss: 0.7893
Epoch 8/50
8/8 [==============================] - 1s 108ms/step - loss: 0.7664
Epoch 9/50
8/8 [==============================] - 1s 118ms/step - loss: 0.7359
Epoch 10/50
8/8 [==============================] - 1s 106ms/step - loss: 0.7112
Epoch 11/50
8/8 [==============================] - 1s 107ms/step - loss: 0.6916
Epoch 12/50
8/8 [==============================] - 1s 115ms/step - loss: 0.6650
Epoch 13/50
8/8 [==============================] 

In [ ]:
# 내일의 수익률과 주가를 예측한다.
px_lstm = np.array(df.tail(20)).reshape(1, t_step, n_feat)
px_cnn = px_lstm.reshape(1, t_step, n_feat, 1)
y_pred = model.predict([px_lstm, px_cnn])[0][0]
y_rtn = y_pred * rtn_std + rtn_mean

if y_rtn > 0:
    print("내일은 {:.2f}% 상승할 것으로 예측됩니다.".format(y_rtn * 100))
else:
    print("내일은 {:.2f}% 하락할 것으로 예측됩니다.".format(y_rtn * 100))
print("예상 주가 = {:.0f}".format(last_price * (1 + y_rtn)))

내일은 0.26% 상승할 것으로 예측됩니다.
예상 주가 = 80204
